# 1. Get query and docuument encoders

In [5]:
import numpy as np

In [6]:
from sentence_transformers import SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

query_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

document_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# Sentences we want to encode. Example:

# Sentences are encoded by calling model.encode()

# 2a get dataset

In [7]:
from datasets import load_dataset

In [8]:
dataset = load_dataset(
    "ms_marco", "v2.1"
)

train_dataset = dataset["train"]

Generating test split: 100%|██████████| 101092/101092 [00:01<00:00, 81164.25 examples/s]


In [9]:
def convert_to_query_pairs(row):
    pairs = []
    for i in range(len(row["passages"]["passage_text"])):
        pairs.append({"query": row["query"], "passage": row["passages"]["passage_text"][i]})
        
    return pairs

In [10]:

# pairs =  convert_to_query_pairs(one_dataset[0])

passages = []
queries = []

filtered_dataset = [item for item in train_dataset if len(item["wellFormedAnswers"]) > 0]

for i in range(len(filtered_dataset)):
    queries.append(filtered_dataset[i]["query"])
    for passage in filtered_dataset[i]["passages"]["passage_text"]:
        passages.append(passage)
        

In [11]:
document_embeddings = document_model.encode(passages)

In [12]:
import pickle
with open("my-embeddings.pkl", "wb") as fOut:
    pickle.dump({'sentences': passages, 'embeddings': document_embeddings},fOut)


In [13]:
document_dim = document_model.get_sentence_embedding_dimension()
query_dim = query_model.get_sentence_embedding_dimension()

# Apply FAISS


In [14]:
import faiss

In [15]:
# IndexFlatL2

index = faiss.IndexFlatL2(document_dim)

index.is_trained

True

In [16]:
index.add(document_embeddings)

In [17]:
index.ntotal

1533292

In [20]:
k = 10

xq = query_model.encode(['Who is JFK'])

D, I = index.search(xq, k)

print(I)

[[ 174926  567411  612022 1121248 1469728  903413  496485  496492  825515
   898918]]


In [21]:
[f'{i}: {passages[i]}' for i in I[0]]

['174926: John Fitzgerald Jack Kennedy (May 29, 1917 – November 22, 1963), commonly referred to by his initials JFK, was an American politician who served as the 35th President of the United States from January 1961 until his assassination in November 1963.',
 '567411: John Fitzgerald Jack Kennedy (May 29, 1917 – November 22, 1963), commonly referred to by his initials JFK, was an American politician who served as the 35th President of the United States from January 1961 until his assassination in November 1963.',
 '612022: John Fitzgerald Jack Kennedy (May 29, 1917 – November 22, 1963), commonly referred to by his initials JFK, was an American politician who served as the 35th President of the United States from January 1961 until his assassination in November 1963.',
 '1121248: John Fitzgerald Jack Kennedy (May 29, 1917 – November 22, 1963), commonly referred to by his initials JFK, was an American politician who served as the 35th President of the United States from January 1961 unt